# Install libraries with given versions

In [0]:
!pip install PyDrive
!pip install keras==2.2.4
!pip install tensorflow==1.13.1
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

# Import Libraries

In [0]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from scipy.stats import cauchy 
from scipy import stats
import time
import operator
import random

# Construct the RBF Network


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1EWNv4GOWsgGXxlkMaTVJMjEAM5jaqRse"}) 
downloaded.GetContentFile('rbfnn.h5')   

In [0]:
np.random.seed(0)
num_lm0 = 100
num_lm = num_lm0 * 10
init_wgt = np.zeros(((1000, 784)))

inp = Input(shape=(28*28,), name='inp')
oup = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp)
oup = Dense(10, activation='softmax')(oup)
model = Model(inp, oup)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("rbfnn.h5")

# Load the Data

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def prepare_data(X,y):
  X = X.reshape((X.shape[0], -1))
  X_sc = X / 256.0
  y_cat = to_categorical(y)
  return X_sc, y_cat

In [0]:
#Example usage
#X_train, y_train = prepare_data(X_train,y_train)
#model.predict(X_train)

# Differential Evolution

In [0]:
def euclidean(s):
    # https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
    # pass s=a-b as argument to the function
    return np.linalg.norm(s)

In [0]:
#Fitness function
def fitness_func(individual,img):
  return (-1)*euclidean(individual-img)

#Given the population and img(whihc is necessary to calculate the fitness function)
#choose one individual whose fitness value is in top p percent.
def get_top_p(population,p,img):
  num_best = int(len(population)*p)
  fitness =  [(individual,fitness_func(individual,img)) for individual in population]
  fitness.sort(key=lambda x: x[1])
  fitness = fitness[-num_best:]
  return random.choice(fitness)[0]
  

## Initialization


In [0]:
#Set seeds
random.seed(0)
np.random.seed(0)
#Image to be evolved
img = X_test[0].reshape((28*28, -1))
imsize = 28*28
#Mean and std for Gaussian noise
mean = 0.0  
std = 1.0  
#Range
R_min = 0.0
R_max = 255.0
#Maximum number of function evaluations
nfe_max = 1000
nfe = 0
#Initial population size
PS_ini = 1000
PS_min = 4
#Pivot for dynamic population change
x = (1/3)*nfe_max
y = (2/3)*PS_ini
PS = PS_ini
#Number of groups
K = 7
#probability of belonging to a group
p_k = np.ones((K,1))*(1/K)
#ratio of the external archive size to the whole population PS
rarc = 1.6
#archive
A_size = int(PS*rarc)
A = np.zeros((A_size,imsize))
A_ind = 0
#scale factor
f = np.ones((PS,1)) 
mu_f = np.ones((K,1)) *0.5
#crossover probability
cr = np.ones((PS,1)) 
mu_cr = np.ones((K,1)) *0.5
#percentage p when we select an individual among top p fits
p = 0.11
#num gen
G = 1
#Initial population
population = np.zeros((PS,imsize))
#Fitness functions
fitness_ind = np.zeros((PS,1))
fitness_U = np.zeros((PS,1))
#donor and trial vectors
V = np.zeros((PS,imsize))
U = np.zeros((PS,imsize))
for i in range(PS):
  noisy_img = img + np.random.normal(mean, std, (imsize,1))
  population[i] = np.clip(noisy_img, 0, 255).reshape(imsize)
X_best = None
X_best_fitness = None

The number of loops may be reduced but we wont do that in order not to deviate from the pseudocode given in paper.

## Main Loop


In [67]:
nfe = 0
start = time.time()

while nfe < nfe_max:
  #############################INITIALIZATION###################################
  #Sets of successfull f and cr values
  #with the corresponding weights
  s_f = [[] for i in range(K)]
  s_f_weights = [[] for i in range(K)]
  s_cr = [[] for i in range(K)]
  s_cr_weights = [[] for i in range(K)]
  #the selected k values for this iteration
  k_selections = np.zeros((K,PS))
  #how many times the specific k value was a success/failure
  k_success = np.zeros((K,1))
  k_failure = np.zeros((K,1))

  ####################GENERATE X_p_best,X_r_1,X_hat_r_2#########################
  #The for loop below is included in the pseducode but will not be included here, 
  #as the same operation is being done below, and the variables are not used 
  #before the repetition.
  '''
  for i in range(PS):
    #X_p_best denotes a certain vector selected randomly from the top 100·p% 
    #individuals of the current population 
    X_p_best =  get_top_p(population,p,img).reshape((imsize,1))
    #X_r_1 denotes a random vector selection from the current population
    X_r_1 = random.choice(population).reshape((imsize,1))
    #X_hat_r_2 denotes a randomly selected solution from the union A and P
    X_hat_r_2 = random.choice(np.concatenate((population, A), axis=0)).reshape((imsize,1))
  '''

  ####################ADJUST POPULATION AND A###################################
  #Adjustment done due to keep the individuals' values in range.
  #and to reduce the array sizes. However, even if the population size gets 
  #smaller than PS, since the loops are in range(PS), the individuals stored in 
  #the population vector in indices > PS will not be reached. 
  #Instead of picking discarded individuals randomly, we shuffle the population
  #and archive array. It should amount to picking random.
  if G > 2:
    #Adjust the individuals of the population; 
    for i in range(PS):
      population[i] = np.clip(population[i], R_min, R_max).reshape(imsize)
    #Adjust storage A; 
    for i in range(A_size):
      A[i] = np.clip(A[i], R_min, R_max).reshape(imsize)
    np.random.shuffle(A)
    np.random.shuffle(population)
    
  #############################SELECT K,F,CR'S##################################  
  #We select a k value for each individual in the population
  #with respect to the selection probabilities (p_k) of each k.
  custm = stats.rv_discrete(name='custm', values=(np.arange(K).reshape(K,1), p_k))
  k_selections = custm.rvs(size=PS)
  #Get f and cr values for each individual
  for i in range(PS):
    cr[i]  = np.random.normal(mu_cr[k_selections[i]], 0.1, 1)
    if cr[i] < 0:
      cr[i] = 0
    f[i] = cauchy.rvs(mu_f[k_selections[i]],0.1,1)
    while f[i] <= 0:
      f[i] = cauchy.rvs(mu_f[k_selections[i]],0.1,1)
    if f[i] > 1:
      f[i] = 1

  ####################GENERATE U,V AND CALCULATE FITNESS########################
  #Generate donor and trial (U and V) vectors
  #Calculate the fitness values
  for i in range(PS):
    X_p_best =  get_top_p(population[:PS],p,img).reshape((imsize,1))
    #X_r_1 denotes a random vector selection from the current population
    X_r_1 = random.choice(population[:PS]).reshape((imsize,1))
    #X_hat_r_2 denotes a randomly selected solution from the union A and P
    X_hat_r_2 = random.choice(np.concatenate((population[:PS], A[:A_size]), axis=0)).reshape((imsize,1))
    #mutation, generate trial vector
    V[i] = (population[i].reshape((imsize,1)) + f[i]*(X_p_best - population[i].reshape((imsize,1)))
           + f[i]*(X_r_1 - X_hat_r_2)).reshape(imsize)
    #crossover, generate donor vector
    U[i] = population[i]
    for j in range(imsize):
      if random.uniform(0, 1) <= cr[i]:
        U[i][j] = V[i][j]
    fitness_ind[i] = fitness_func(population[i],img)
    fitness_U[i] = fitness_func(U[i],img)
  
  ################################INCREMENT nfe#################################
  #Increment nfe
  nfe += PS

  #####################DETERMINE UN/SUCCESSFULL#################################
  #Determine the successful and unsuccessful cr,f,k values.
  #Add unsuccessful individuals to the archive
  for i in range(PS):
    k = k_selections[i]
    if fitness_ind[i] <= fitness_U[i]:
      s_cr[k].append(cr[i])
      s_cr_weights[k].append(np.std(np.subtract(population[i],U[i])))
      s_f[k].append(f[i])
      s_f_weights[k].append(fitness_U[i]-fitness_ind[i])
      population[i] = U[i]
      k_success[k] += 1
    else:
      if A_ind >= A_size:
        A_ind = 0
      A[A_ind] = population[i]
      A_ind += 1
      k_failure[k] += 1

  ######################UPDATE MU_CR AND MU_F's#################################
  #update params (mixed with Jade-like update)
  for k in range(K):
    if len(s_cr[k]) != 0:
      s_cr_weights[k] = s_cr_weights[k]/np.sum(s_cr_weights[k])
      s_f_weights[k] = s_f_weights[k]/np.sum(s_f_weights[k])
      mean_cr = np.sum(np.multiply(np.square(s_cr[k]),s_cr_weights[k]))/np.sum(np.multiply(s_cr[k],s_cr_weights[k]))
      mean_f = np.sum(np.multiply(np.square(s_f[k]),s_f_weights[k]))/np.sum(np.multiply(s_f[k],s_f_weights[k]))  
      #update mu_cr
      if (np.max(s_cr[k]) > 0) and (mu_cr[k] != 0):
        mu_cr[k] = mean_cr
      #update mu_f
      mu_f[k] = mean_f
  
  ###########################UPDATE GROUP PROBS#################################
  #Update group probabilities
  for k in range(K):
    if k_success[k] != 0:
      k_success[k] = np.square(k_success[k])/(np.sum(k_success[k])*(k_success[k]+k_failure[k]))
    else:
      p_k[k] = 0.000000001
  p_k = p_k/np.sum(p_k)

  ###########################END ITERATION######################################
  #increment generation count
  G += 1
  #Get the best individual
  X_best = get_top_p(population[:PS],1/(len(population[:PS])),img)
  #Get the fitness of best individual
  X_best_fitness = fitness_func(X_best,img)
  #Give feedback
  print(nfe, " : "  ,X_best_fitness,PS)

  ######################ADJUST POPULATION SIZE##################################
  #adjust population size 
  #else part changed. there is something wrong with the formula on the paper.
  #since the ends of the two partial functions do not meet, when nfe > x the 
  #population increases. change is to fix this.
  if nfe <= x:
    PS = int(np.ceil((((y-PS_ini)/np.square(x-PS_ini))*np.square(nfe-PS_ini)) + PS_ini))
  else:
    initial_formula = int(np.floor((((y-PS_ini)/(x-PS_min))*(nfe-nfe_max)) + PS_min))
    if initial_formula < PS:
      PS = initial_formula
  #adjust archive size according to the new population
  A_size = int(rarc*PS)
  #Note: archive update moved within "Determine un/successful" as the computation
  #would be too redundant.  
end = time.time()
print("Time: ",end-start ," seconds")

1000  :  -72987.32631928888 1000
Time:  2017.5735175609589  seconds


# JADE-like parameter update

I felt like this has faster convergence with same parameter settings.


In [27]:
#A constant used in JADE. Included here as the update is 
#mixed between the paper and jade
c = 0.1
nfe = 0
start = time.time()
while nfe < nfe_max:
  #############################INITIALIZATION###################################
  #Sets of successfull f and cr values
  #with the corresponding weights
  s_f = []
  s_f_weights = []
  s_cr = []
  s_cr_weights = []
  #the selected k values for this iteration
  k_selections = np.zeros((K,1))
  #how many times the specific k value was a success/failure
  k_success = np.zeros((K,1))
  k_failure = np.zeros((K,1))

  ####################GENERATE X_p_best,X_r_1,X_hat_r_2#########################
  #The for loop below is included in the pseducode but will not be included here, 
  #as the same operation is being done below, and the variables are not used 
  #before the repetition.
  '''
  for i in range(PS):
    #X_p_best denotes a certain vector selected randomly from the top 100·p% 
    #individuals of the current population 
    X_p_best =  get_top_p(population,p,img).reshape((imsize,1))
    #X_r_1 denotes a random vector selection from the current population
    X_r_1 = random.choice(population).reshape((imsize,1))
    #X_hat_r_2 denotes a randomly selected solution from the union A and P
    X_hat_r_2 = random.choice(np.concatenate((population, A), axis=0)).reshape((imsize,1))
  '''

  ####################ADJUST POPULATION AND A###################################
  #Adjustment done due to keep the individuals' values in range.
  #and to reduce the array sizes. However, even if the population size gets 
  #smaller than PS, since the loops are in range(PS), the individuals stored in 
  #the population vector in indices > PS will not be reached. 
  #Instead of picking discarded individuals randomly, we shuffle the population
  #and archive array. It should amount to picking random.
  if G > 2:
    #Adjust the individuals of the population; 
    for i in range(PS):
      population[i] = np.clip(population[i], R_min, R_max).reshape(imsize)
    #Adjust storage A; 
    for i in range(A_size):
      A[i] = np.clip(A[i], R_min, R_max).reshape(imsize)
    np.random.shuffle(A)
    np.random.shuffle(population)
    
  #############################SELECT K,F,CR'S##################################  
  #We select a k value for each individual in the population
  #with respect to the selection probabilities (p_k) of each k.
  custm = stats.rv_discrete(name='custm', values=(np.arange(K).reshape(K,1), p_k))
  k_selections = custm.rvs(size=PS)
  #Get f and cr values for each individual
  for i in range(PS):
    cr[i]  = np.random.normal(mu_cr[k_selections[i]], 0.1, 1)
    if cr[i] < 0:
      cr[i] = 0
    f[i] = cauchy.rvs(mu_f[k_selections[i]],0.1,1)
    while f[i] <= 0:
      f[i] = cauchy.rvs(mu_f[k_selections[i]],0.1,1)
    if f[i] > 1:
      f[i] = 1

  ####################GENERATE U,V AND CALCULATE FITNESS########################
  #Generate donor and trial (U and V) vectors
  #Calculate the fitness values
  for i in range(PS):
    X_p_best =  get_top_p(population[:PS],p,img).reshape((imsize,1))
    #X_r_1 denotes a random vector selection from the current population
    X_r_1 = random.choice(population[:PS]).reshape((imsize,1))
    #X_hat_r_2 denotes a randomly selected solution from the union A and P
    X_hat_r_2 = random.choice(np.concatenate((population[:PS], A[:A_size]), axis=0)).reshape((imsize,1))
    #mutation, generate trial vector
    V[i] = (population[i].reshape((imsize,1)) + f[i]*(X_p_best - population[i].reshape((imsize,1)))
           + f[i]*(X_r_1 - X_hat_r_2)).reshape(imsize)
    #crossover, generate donor vector
    U[i] = population[i]
    for j in range(imsize):
      if random.uniform(0, 1) <= cr[i]:
        U[i][j] = V[i][j]
    fitness_ind[i] = fitness_func(population[i],img)
    fitness_U[i] = fitness_func(U[i],img)
  
  ################################INCREMENT nfe#################################
  #Increment nfe
  nfe += PS

  #####################DETERMINE UN/SUCCESSFULL#################################
  #Determine the successful and unsuccessful cr,f,k values.
  #Add unsuccessful individuals to the archive
  for i in range(PS):
    k = k_selections[i]
    if fitness_ind[i] <= fitness_U[i]:
      s_cr.append(cr[i])
      s_cr_weights.append(np.std(np.subtract(population[i],U[i])))
      s_f.append(f[i])
      s_f_weights.append(fitness_U[i]-fitness_ind[i])
      population[i] = U[i]
      k_success[k] += 1
    else:
      if A_ind >= A_size:
        A_ind = 0
      A[A_ind] = population[i]
      A_ind += 1
      k_failure[k] += 1

  ######################UPDATE MU_CR AND MU_F's#################################
  #update params (mixed with Jade-like update)
  if len(s_cr) != 0:
    s_cr_weights = (s_cr_weights/np.sum(s_cr_weights)).reshape((len(s_cr_weights),1))
    s_f_weights = (s_f_weights/np.sum(s_f_weights)).reshape((len(s_f_weights),1))
    mean_cr = np.sum(np.multiply(np.square(s_cr),s_cr_weights))/np.sum(np.multiply(s_cr,s_cr_weights))
    mean_f = np.sum(np.multiply(np.square(s_f),s_f_weights))/np.sum(np.multiply(s_f,s_f_weights))  
    #we can make it more group specific as discussed in the paper
    for i in range(K):
      mu_cr[i] = (1-c)*mu_cr[i] + c*mean_cr
      mu_f[i] = (1-c)*mu_f[i] + c*mean_f
  
  ###########################UPDATE GROUP PROBS#################################
  #Update group probabilities
  for k in range(K):
    if k_success[k] != 0:
      k_success[k] = np.square(k_success[k])/(np.sum(k_success[k])*(k_success[k]+k_failure[k]))
    else:
      p_k[k] = 0.000000001
  p_k = p_k/np.sum(p_k)

  ###########################END ITERATION######################################
  #increment generation count
  G += 1
  #Get the best individual
  X_best = get_top_p(population[:PS],1/(len(population[:PS])),img)
  #Get the fitness of best individual
  X_best_fitness = fitness_func(X_best,img)
  #Give feedback
  print(nfe, " : "  ,X_best_fitness,PS)

  ######################ADJUST POPULATION SIZE##################################
  #adjust population size 
  #else part changed. there is something wrong with the formula on the paper.
  #since the ends of the two partial functions do not meet, when nfe > x the 
  #population increases. change is to fix this.
  if nfe <= x:
    PS = int(np.ceil((((y-PS_ini)/np.square(x-PS_ini))*np.square(nfe-PS_ini)) + PS_ini))
  else:
    initial_formula = int(np.floor((((y-PS_ini)/(x-PS_min))*(nfe-nfe_max)) + PS_min))
    if initial_formula < PS:
      PS = initial_formula
  #adjust archive size according to the new population
  A_size = int(rarc*PS)
  #Note: archive update moved within "Determine un/successful" as the computation
  #would be too redundant.  
end = time.time()
print("Time: ",end-start ," seconds")

25  :  -73059.76546730741 25
50  :  -73035.68653107567 25
75  :  -73018.66123040198 25
100  :  -73011.64879336844 25
125  :  -73004.16452333861 25
150  :  -72982.25404294368 25
175  :  -72959.08576276507 25
200  :  -72918.72971503744 25
225  :  -72888.90738921984 25
250  :  -72831.69508205332 25
275  :  -72763.96083924644 25
300  :  -72692.94043304944 25
325  :  -72585.42758244676 25
350  :  -72388.33537501819 25
375  :  -72304.1175371469 25
400  :  -72151.25197852019 25
425  :  -72023.64637070132 25
450  :  -71876.21738175966 25
475  :  -71628.98214665045 25
500  :  -71415.47939820943 25
525  :  -71273.04199158274 25
550  :  -71117.10427948483 25
575  :  -70905.74397800486 25
600  :  -70701.67198358351 25
625  :  -70604.89692345636 25
650  :  -70357.31765447241 25
675  :  -70268.85663196856 25
700  :  -70020.77329211167 25
725  :  -69945.13072394243 25
750  :  -69940.75042731898 25
775  :  -69704.80206191864 25
800  :  -69499.72397748953 25
825  :  -69296.25435311404 25
850  :  -69265

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in true_divide


9520  :  -59858.83738404894 5
9525  :  -59858.57409648848 5
9530  :  -59858.80348153485 5
9535  :  -59858.77931952295 5
9540  :  -59858.57409648848 5
9545  :  -59858.80348153485 5
9550  :  -59858.80348153485 5
9555  :  -59858.914894133195 5
9560  :  -59858.57409648848 5
9565  :  -59858.852249616015 5
9570  :  -59858.76312906434 5
9575  :  -59858.76312906434 5
9580  :  -59858.852249616015 5
9585  :  -59858.87758492486 5
9590  :  -59858.77931952295 5
9595  :  -59858.76312906434 5
9600  :  -59858.90082018455 5
9605  :  -59858.80348153485 5
9609  :  -59858.57409648848 4
9613  :  -59858.77931952295 4
9617  :  -59858.860955499506 4
9621  :  -59858.83738404894 4
9625  :  -59858.91288120394 4
9629  :  -59858.852249616015 4
9633  :  -59858.76312906434 4
9637  :  -59858.57409648848 4
9641  :  -59858.76312906434 4
9645  :  -59858.81838065673 4
9649  :  -59858.57409648848 4
9653  :  -59858.77931952295 4
9657  :  -59858.77931952295 4
9661  :  -59858.852249616015 4
9665  :  -59858.76312906434 4
9669

# Visualize results


In [0]:
#Truth:7
y_test[0]

In [0]:
#Model predicts: 7
X_test_sc = X_test/255.0
preds = model.predict(X_test_sc[0].reshape((1,28*28)))
np.argmax(preds,axis=1)

In [0]:
X_best_sc = X_best / 255.0

In [0]:
#Predicts evolved example:7
model.predict(X_best_sc.reshape((1,28*28)))

In [0]:
plt.imshow(X_best.reshape((28,28)))

In [0]:
plt.imshow(X_test[0].reshape((28,28)))

In [0]:
plt.imshow((X_best/255.0).reshape((28,28)))

In [0]:
plt.imshow((X_test[0]/255.0).reshape((28,28)))